In [ ]:
# Newton Rhapson method 1

import sys, numpy as np, matplotlib.pyplot as plt

s = 0.6 # the bargaining power of the workers
psi_e = 0
psi_u = 0.5
psi_a = 1
n = 1
y_u = 0.4
y_a = 0.4
bar_p = 1
c = 0.2

# let the matching function in labor market 
# m(u,v)=kappa*u**zeta*v**(1-zeta) 
# then q(theta) = m(u,v)/v = kappa*theta**(-zeta),f(theta) = m(u,v)/u = kappa*theta**(1-zeta) 

kappa = 0.5
zeta = 0.35

'''
q = kappa* (np.abs(theta)) ** (-zeta)
f = theta*q
b = (n-u)*(1+psi_e)+u*(1+psi_u)+a*(1+psi_a)
w = (1-s)*y_u + s*pi
    
c-q*(1-s)*(pi-y_u), 
(n-u)*(1+psi_e)/b*(1-2*psi_e/(1+psi_e)*(n-u)/b
                  )*w/bar_p*(bar_p-c)
+u*(1+psi_u)/b*(1-2*psi_u/(1+psi_u)*(n-u)/b
                  )*y_u/bar_p*(bar_p-c)
+a*(1+psi_a)/b*(1-2*psi_a/(1+psi_a)*(n-u)/b
                  )*y_a/bar_p*(bar_p-c)
-pi,
u-n*(1-f)+(n-u)*s


'''

def func1(x):
    return c-kappa*x[1]**(-zeta)*(1-s)*(x[2]-y_u)

def func2(x):
    b = (n-x[0])*(1+psi_e)+x[0]*(1+psi_u)+a*(1+psi_a)
    w = (1-s)*y_u + s*x[2]
    return (n-x[0])*(1+psi_e)/b*(1-2*psi_e/(1+psi_e)*(n-x[0])/b)*w/bar_p*(bar_p-c)+x[0]*(1+psi_u)/b*(1-2*psi_u/(1+psi_u)*(n-x[0])/b)*y_u/bar_p*(bar_p-c)+a*(1+psi_a)/b*(1-2*psi_a/(1+psi_a)*(n-x[0])/b)*y_a/bar_p*(bar_p-c)-x[2]

def func3(x):
    return x[0]-n*(1-kappa*x[1]**(1-zeta))-(n-x[0])*s

func = [func1, func2, func3]

# finds the Jacob matrix of a system of equations
def jacob_mat(x, func=func, functol=1e-4):
    n = len(x)
    m = len(func)
    jacob_matrix = np.zeros([m, n])
    for iterfor0 in range(m):
        for iterfor1 in range(n):
            xtrial0 = x.copy()
            xtrial0[iterfor1] -= functol
            xtrial1 = x.copy()
            xtrial1[iterfor1] += functol
            jacob_matrix[iterfor0, iterfor1] = (func[iterfor0](xtrial1) - func[iterfor0](xtrial0)) / (2.0 * functol)
    return jacob_matrix

def newton(seed, func=func, tol=1e-6, functol=1e-8, damping=1, maxiter=1000):
    """
    Function that finds the roots of func using Newton-Raphson's method 
    By default, the derivative is found using 3-point numeric differentiation
    but user can input a function in argument dfuncdx to use it to evaluate
    exact derivative
    Optional input arguments are tol=1e-6, functol=1e-8, damping=1, maxiter=1000
    Output arguments are: 
        - x1: the final roots found
        - dev: the relative errors of the roots between last 2 iterations
        - backupx0: for graphing purposes it saves root trials each iteration
        - backupdev: errors in every iteration
        - backupdx: correction factors every iteration
        - itera: number of iterations
    """
    itera = 0
    seed = np.atleast_1d(seed)
    x0 = seed # initial guess
    x0 = x0.astype(np.float64)
    n = len(seed) # number of roots
    m = len(func)
    dev = 10**4
    backupx0 = np.empty([0, n])

    while np.any(dev > tol) and (itera < maxiter):
        itera += 1
        funvectorial = np.zeros(n)
        for iterfor2 in range(m):
            funvectorial[iterfor2] = -func[iterfor2](x0)
        jacob = jacob_mat(x0, func, functol)
        
        dx = damping * np.linalg.solve(jacob, funvectorial)
        x1 = x0 + dx
        dev = abs(x0 - x1) / abs(x0)  # error
        backupx0 = np.append(backupx0, x0)
        x0 = x1
    backupx0 = np.append(backupx0, x0)
    backupx0 = backupx0.reshape([itera + 1, n])
    
    return x1, itera, dev, backupx0

x11, iter1, dev1, backupx01 = newton(seed, func=func)

a = 0.4
seed = [1, 1, 1]
x0 = seed
functol=1e-8
newton(seed, func=func)

In [ ]:
# Newton Rhapson method 2
# multivariate Newton Rhapson method to solve nonlinear system
import autograd.numpy as np
from autograd import grad, jacobian


func1 = lambda x: c-kappa*x[1]**(-zeta)*(1-s)*(x[2]-y_u)
#b = (n-x[0])*(1+psi_e)+x[0]*(1+psi_u)+a*(1+psi_a)
# w = (1-s)*y_u + s*x[2]
func2 = lambda x: (n-x[0])*(1+psi_e)/((n-x[0])*(1+psi_e)+x[0]*(1+psi_u)\
                                      +a*(1+psi_a))*(1-2*psi_e/(1+psi_e)*(n-x[0])/((n-x[0])*(1+psi_e)+x[0]*(1+psi_u)+a*(1+psi_a)))\
*((1-s)*y_u + s*x[2])/bar_p*(bar_p-c)+x[0]*(1+psi_u)/((n-x[0])*(1+psi_e)+x[0]*(1+psi_u)+a*(1+psi_a))\
*(1-2*psi_u/(1+psi_u)*(n-x[0])/((n-x[0])*(1+psi_e)+x[0]*(1+psi_u)+a*(1+psi_a)))\
*y_u/bar_p*(bar_p-c)+a*(1+psi_a)/((n-x[0])*(1+psi_e)+x[0]*(1+psi_u)+a*(1+psi_a))\
*(1-2*psi_a/(1+psi_a)*(n-x[0])/((n-x[0])*(1+psi_e)+x[0]*(1+psi_u)+a*(1+psi_a)))*y_a/bar_p*(bar_p-c)-x[2]
func3 = lambda x: x[0]-n*(1-kappa*x[1]**(1-zeta))-(n-x[0])*s

#for i in range(3):
jac_func1 = jacobian(func1)
jac_func2 = jacobian(func2)
jac_func3 = jacobian(func3)

i = 0
error =100
tol = 10**(-4)
maxiter = 10**2
M = 3
N = 3
x_0 = np.array([20,20,10],dtype=float).reshape(N,1)

unemp = []
tight_labor = []
net_profit = []
aging = []

for a in np.arange(0.01, 1, 0.05):
    while np.any(abs(error) > tol) and i < maxiter:
        fun_evaluate = np.array([func1(x_0), func2(x_0), func3(x_0)]).reshape(M,1)
        flat_x_0 = x_0.flatten()

        jac = np.array([jac_func1(flat_x_0), jac_func2(flat_x_0),jac_func3(flat_x_0)])
        jac = jac.reshape(N,M)

        x_new = x_0 - np.linalg.inv(jac)@fun_evaluate

        error = x_new - x_0

        x_0 = x_new
        print(i)
        print(x_new)
        
        i += 1

    if x_new[0] > 0:
        unemp.append(x_new[0])
        tight_labor.append(x_new[1])
        net_profit.append(x_new[2])
        aging.append(a)

#aging = np.arange(0.01, 1, 0.1)
plt.figure()
#plt.subplot(211)
plt.plot(aging, unemp)
plt.xlabel('Aged Dependency Ratio')
plt.ylabel('Unemployment Rate')

plt.show()

In [ ]:
# Newton Rhapson method 3

import numpy as np 
from numpy.linalg import inv
import matplotlib.pyplot as plt 

def f1(y1_old,y1,y2,y3,dt):
    return y1_old + dt*(-0.04*y1 + y2*y3*pow(10,4)) - y1
def f2(y2_old,y1,y2,y3,dt):
    return y2_old + dt*(0.04*y1 - pow(10,4)*y2*y3 - 3*pow(10,7)*pow(y2,2)) -y2
def f3(y3_old,y1,y2,y3,dt):
    return y3_old + dt*(3*pow(10,7)*pow(y2,2)) - y3
"""
Jacobian matrix should be solved numerically
"""
def jacobian(y1_old,y2_old,y3_old,y1,y2,y3,dt):
    J = np.ones((3,3))
    h=1e-8
    #row 1
    J[0,0] = (f1(y1_old,y1+h,y2,y3,dt)-f1(y1_old,y1,y2,y3,dt))/h
    J[0,1] = (f1(y1_old,y1,y2+h,y3,dt)-f1(y1_old,y1,y2,y3,dt))/h
    J[0,2] = (f1(y1_old,y1,y2,y3+h,dt)-f1(y1_old,y1,y2,y3,dt))/h

    #row2
    J[1,0] = (f2(y2_old,y1+h,y2,y3,dt)-f2(y2_old,y1,y2,y3,dt))/h
    J[1,1] = (f2(y2_old,y1,y2+h,y3,dt)-f2(y2_old,y1,y2,y3,dt))/h
    J[1,2] = (f2(y2_old,y1,y2,y3+h,dt)-f2(y2_old,y1,y2,y3,dt))/h

    #row3
    J[2,0] = (f3(y3_old,y1+h,y2,y3,dt)-f3(y3_old,y1,y2,y3,dt))/h
    J[2,1] = (f3(y3_old,y1,y2+h,y3,dt)-f3(y3_old,y1,y2,y3,dt))/h
    J[2,2] = (f3(y3_old,y1,y2,y3+h,dt)-f3(y3_old,y1,y2,y3,dt))/h

    return J


#intial conditions
dt = 0.5 #step size
y1 = 1
y2 = 0
y3 = 0


#y_new = y_guess -alpha*inv(J)*F
y_new = np.ones((3,1))
y_old = np.ones((3,1))
y_guess = np.ones((3,1))

#conditions for the time array

t= np.arange(0,600,dt) 
total_iter=len(t)
y=np.ones((3,total_iter))

#numpy column matrix for the F
F = np.copy(y_guess)


time_iter = 0
iter =0
print(y)
print(total_iter)

#outer time integration loop
for i in range(0,total_iter):

    y_old[0] = y1
    y_old[1] = y2
    y_old[2] = y3

    y_guess[0] = y1
    y_guess[1] = y2 
    y_guess[2] = y3 

    alpha = 1
    tol = 1e-8
    error = 9e9


    # newton raphson loop
    while error>tol:

        F[0] = f1(y_old[0],y_guess[0],y_guess[1], y_guess[2],dt)
        F[1] = f2(y_old[1],y_guess[0],y_guess[1],y_guess[2],dt)
        F[2] = f3(y_old[2],y_guess[0],y_guess[1],y_guess[2],dt)

        J = jacobian(y_old[0],y_old[1],y_old[2],y_guess[0],y_guess[1],y_guess[2],dt)

        y_new = y_guess-alpha*np.matmul(inv(J),F)
        error = np.max(np.abs(y_guess - y_new))

        y_guess = y_new
        log_message = 'iteration # = {0} y1 = {1} y2 = {2} y3 = {3}'.format(iter,y_new[0],y_new[1],y_new[2]) 
        print(log_message)
        iter = iter + 1

    # updating the new values.
    y1 = y_new[0]
    y2 = y_new[1]
    y3 = y_new[2]

    y[0,time_iter] = y_new[0]
    y[1,time_iter] = y_new[1]
    y[2,time_iter] = y_new[2]


    time_iter = time_iter + 1

plt.plot(t,y[0])
plt.plot(t,y[1])
plt.plot(t,y[2])
plt.legend(['Y1','Y2','Y3'])
plt.show()